In [ ]:
open System

let private random = Random ()

let rollD6 () = random.Next (1, 7)

In [ ]:
let numDices log max =
    let rec numDices' n p =
        if log then printfn $"numDices / n: {n} / max: {max} / p: {p}"
        if p >= max
        then n
        else numDices' (n + 1) (p * 6)
    numDices' 0 1

In [ ]:
let pow6 = 1 |> Seq.unfold (fun state -> Some (state, state * 6)) |> Seq.cache
pow6

[ 1, 6, 36, 216, 1296, 7776, 46656, 279936, 1679616, 10077696, 60466176, 362797056, -2118184960, 175792128, 1054752768, 2033549312, -683606016, 193331200, 1159987200, -1630011392 ... (more) ]

In [ ]:
let rec rollAcc log rolls power acc =
    match rolls with
    | _ when power < 0 ->
        if log then printfn $"rollAcc / power: {power} / acc: {acc}"
        Some (acc, rolls)
    | [] -> None
    | roll :: rest when roll > 1 ->
        let coeff = pow6 |> Seq.item power
        let value = (roll - 1) * coeff
        if log then printfn $"rollAcc / power: {power} / acc: {acc} / roll: {roll} / value: {value}"
        rollAcc log rest (power - 1) (acc + value)
    | roll :: rest ->
        if log then printfn $"rollAcc / power: {power} / acc: {acc} / roll: {roll}"
        rollAcc log rest (power - 1) acc

let fixedRoll log max rolls =
    let rec rollMax power =
        match rollAcc log rolls power 0 with
        | Some (result, _) when result >= 1 && result <= max -> Some result
        | _ -> None

    rollMax (List.length rolls - 1)

let progressiveRoll log reroll max =
    let rec rollMax power =
        let rec loop rolls size =
            if size < power + 1
            then loop (rollD6 () :: rolls) (size + 1)
            else
                match rollAcc log rolls power 0 with
                | Some (result, _) when result >= 1 && result <= max -> result
                | _ when reroll -> loop (List.init power (fun _ -> rollD6 ())) power
                | _ -> loop (rollD6 () :: rolls) (size + 1)
        loop [] 0
    rollMax ((numDices log max) - 1)

In [ ]:
//// ignore

progressiveRoll true false 2000

732

numDices / n: 0 / max: 2000 / p: 1
numDices / n: 1 / max: 2000 / p: 6
numDices / n: 2 / max: 2000 / p: 36
numDices / n: 3 / max: 2000 / p: 216
numDices / n: 4 / max: 2000 / p: 1296
numDices / n: 5 / max: 2000 / p: 7776
rollAcc / power: 4 / acc: 0 / roll: 3 / value: 2592
rollAcc / power: 3 / acc: 2592 / roll: 3 / value: 432
rollAcc / power: 2 / acc: 3024 / roll: 1
rollAcc / power: 1 / acc: 3024 / roll: 4 / value: 18
rollAcc / power: 0 / acc: 3042 / roll: 4 / value: 3
rollAcc / power: -1 / acc: 3045
rollAcc / power: 4 / acc: 0 / roll: 4 / value: 3888
rollAcc / power: 3 / acc: 3888 / roll: 3 / value: 432
rollAcc / power: 2 / acc: 4320 / roll: 3 / value: 72
rollAcc / power: 1 / acc: 4392 / roll: 1
rollAcc / power: 0 / acc: 4392 / roll: 4 / value: 3
rollAcc / power: -1 / acc: 4395
rollAcc / power: 4 / acc: 0 / roll: 1
rollAcc / power: 3 / acc: 0 / roll: 4 / value: 648
rollAcc / power: 2 / acc: 648 / roll: 3 / value: 72
rollAcc / power: 1 / acc: 720 / roll: 3 / value: 12
rollAcc / power: 0 /

In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> progressiveRoll false true 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 9) Item1 1 Item2 9 1 (2, 13) Item1 2 Item2 13 2 (3, 9) Item1 3 Item2 9 3 (4, 11) Item1 4 Item2 11 4 (5, 10) Item1 5 Item2 10 5 (6, 7) Item1 6 Item2 7 6 (7, 11) Item1 7 Item2 11 7 (8, 9) Item1 8 Item2 9 8 (9, 15) Item1 9 Item2 15 9 (10, 6) Item1 10 Item2 6

In [ ]:
let rollMax fn max n =
    [1..n]
    |> List.map (fun _ -> fn max)
    |> List.groupBy id
    |> List.map (fun (_, v) -> v.Length)

In [ ]:
//// ignore

let max = 10
let n = 30
let even = (n / max) |> int

let rec rollN current =
    let roll = rollMax (progressiveRoll false true) max n
    if roll |> List.forall ((=) even)
    then current
    else rollN (current + 1)

rollN 0

48024

In [ ]:
//// ignore

[1..100]
|> List.map (fun i ->
    let roll = rollN 0
    printfn $"i: {i} / roll: {roll}"
    roll
)
|> List.map float
|> List.average

230469.7

i: 1 / roll: 144155
i: 2 / roll: 1027
i: 3 / roll: 220594
i: 4 / roll: 240752
i: 5 / roll: 223283
i: 6 / roll: 505722
i: 7 / roll: 418985
i: 8 / roll: 56371
i: 9 / roll: 16732
i: 10 / roll: 5723
i: 11 / roll: 853
i: 12 / roll: 36888
i: 13 / roll: 168677
i: 14 / roll: 61066
i: 15 / roll: 186089
i: 16 / roll: 138750
i: 17 / roll: 98385
i: 18 / roll: 304317
i: 19 / roll: 115598
i: 20 / roll: 653826
i: 21 / roll: 347823
i: 22 / roll: 52404
i: 23 / roll: 1195484
i: 24 / roll: 420135
i: 25 / roll: 62350
i: 26 / roll: 125886
i: 27 / roll: 38883
i: 28 / roll: 147603
i: 29 / roll: 115186
i: 30 / roll: 448777
i: 31 / roll: 522129
i: 32 / roll: 58804
i: 33 / roll: 8997
i: 34 / roll: 452073
i: 35 / roll: 40493
i: 36 / roll: 7406
i: 37 / roll: 21381
i: 38 / roll: 6979
i: 39 / roll: 139365
i: 40 / roll: 760697
i: 41 / roll: 604148
i: 42 / roll: 113012
i: 43 / roll: 10864
i: 44 / roll: 758050
i: 45 / roll: 202112
i: 46 / roll: 629623
i: 47 / roll: 40512
i: 48 / roll: 15313
i: 49 / roll: 151087
i: 50 

In [ ]:
let max = 2000
let dices = numDices true max
dices

5

numDices / n: 0 / max: 2000 / p: 1
numDices / n: 1 / max: 2000 / p: 6
numDices / n: 2 / max: 2000 / p: 36
numDices / n: 3 / max: 2000 / p: 216
numDices / n: 4 / max: 2000 / p: 1296
numDices / n: 5 / max: 2000 / p: 7776


In [ ]:
fixedRoll true 2000 [1; 5; 4; 4; 5]

Value,994


rollAcc / power: 4 / acc: 0 / roll: 1
rollAcc / power: 3 / acc: 0 / roll: 5 / value: 864
rollAcc / power: 2 / acc: 864 / roll: 4 / value: 108
rollAcc / power: 1 / acc: 972 / roll: 4 / value: 18
rollAcc / power: 0 / acc: 990 / roll: 5 / value: 4
rollAcc / power: -1 / acc: 994


In [ ]:
fixedRoll true 2000 [2; 2; 6; 4; 5]

Value,1714


rollAcc / power: 4 / acc: 0 / roll: 2 / value: 1296
rollAcc / power: 3 / acc: 1296 / roll: 2 / value: 216
rollAcc / power: 2 / acc: 1512 / roll: 6 / value: 180
rollAcc / power: 1 / acc: 1692 / roll: 4 / value: 18
rollAcc / power: 0 / acc: 1710 / roll: 5 / value: 4
rollAcc / power: -1 / acc: 1714


In [ ]:
fixedRoll true 2000 [4; 1; 1; 2; 3]

<null>

rollAcc / power: 4 / acc: 0 / roll: 4 / value: 3888
rollAcc / power: 3 / acc: 3888 / roll: 1
rollAcc / power: 2 / acc: 3888 / roll: 1
rollAcc / power: 1 / acc: 3888 / roll: 2 / value: 6
rollAcc / power: 0 / acc: 3894 / roll: 3 / value: 2
rollAcc / power: -1 / acc: 3896
